# Feature Engineering - House Price Prediction

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Importing the train dataset
dataset=pd.read_csv('train.csv')
dataset.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD         Normal     208500  
1   2007        WD         Normal     181500  
2   2008        WD         Normal     223500  
3   2006        WD        Abnorml     140000  
4   2008        WD         Normal     250000  

[5 rows x 81 columns]

It is always advised to do the test train split which will neglet the risk of data leakage:


In [3]:
# While doing feature engineering it is allways advised to split the data incase you are not having -
# seperate train test data available.
# from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test=train_test_split(dataset,dataset['SalePrice'],test_size=0.1,random_state=0)

## Missing Values:

In [4]:
#Handling the missing categorical features:
cat_features_null=[features for features in dataset.columns if dataset[features].isnull().sum()>1 and dataset[features].dtypes=='O']
for feature in cat_features_null:
    print("Percentage of null present in ",feature," is ",np.round(dataset[feature].isnull().mean(),4))

Percentage of null present in  Alley  is  0.9377
Percentage of null present in  MasVnrType  is  0.0055
Percentage of null present in  BsmtQual  is  0.0253
Percentage of null present in  BsmtCond  is  0.0253
Percentage of null present in  BsmtExposure  is  0.026
Percentage of null present in  BsmtFinType1  is  0.0253
Percentage of null present in  BsmtFinType2  is  0.026
Percentage of null present in  FireplaceQu  is  0.4726
Percentage of null present in  GarageType  is  0.0555
Percentage of null present in  GarageFinish  is  0.0555
Percentage of null present in  GarageQual  is  0.0555
Percentage of null present in  GarageCond  is  0.0555
Percentage of null present in  PoolQC  is  0.9952
Percentage of null present in  Fence  is  0.8075
Percentage of null present in  MiscFeature  is  0.963


In [5]:
# Replacing the missing values in categorical_features with the word 'Missing':
#Function to replace the null values in categorical features:
def replace_cat_features(dataset,cat_feature):
    data=dataset.copy()
    data[cat_feature]=data[cat_feature].fillna('Missing')
    return data
dataset=replace_cat_features(dataset,cat_features_null)
dataset[cat_features_null].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [6]:
dataset.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street    Alley LotShape  \
0   1          60       RL         65.0     8450   Pave  Missing      Reg   
1   2          20       RL         80.0     9600   Pave  Missing      Reg   
2   3          60       RL         68.0    11250   Pave  Missing      IR1   
3   4          70       RL         60.0     9550   Pave  Missing      IR1   
4   5          60       RL         84.0    14260   Pave  Missing      IR1   

  LandContour Utilities  ... PoolArea   PoolQC    Fence MiscFeature MiscVal  \
0         Lvl    AllPub  ...        0  Missing  Missing     Missing       0   
1         Lvl    AllPub  ...        0  Missing  Missing     Missing       0   
2         Lvl    AllPub  ...        0  Missing  Missing     Missing       0   
3         Lvl    AllPub  ...        0  Missing  Missing     Missing       0   
4         Lvl    AllPub  ...        0  Missing  Missing     Missing       0   

  MoSold YrSold  SaleType  SaleCondition  SalePrice  
0      2   2008        WD         Normal     208500  
1      5   2007        WD         Normal     181500  
2      9   2008        WD         Normal     223500  
3      2   2006        WD        Abnorml     140000  
4     12   2008        WD         Normal     250000  

[5 rows x 81 columns]

In [7]:
#Handling numerical features with null values:
num_features_null=[features for features in dataset.columns if dataset[features].isnull().sum()>1 and dataset[features].dtypes!='O']
for feature in num_features_null:
    print("Percentage of null in ",feature," is ",np.round(dataset[feature].isnull().mean(),4))

Percentage of null in  LotFrontage  is  0.1774
Percentage of null in  MasVnrArea  is  0.0055
Percentage of null in  GarageYrBlt  is  0.0555


In [8]:
# Replacing numerical missing values:

for feature in num_features_null:
    # Finding the median of the feature:
    median_feature=dataset[feature].median()
    
    #creating new feature to indicate the missing entries:
    dataset[feature+" Nil"]=np.where(dataset[feature].isnull(),1,0)
    dataset[feature].fillna(median_feature,inplace=True)
    
dataset[num_features_null].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [9]:
# Handling the temperal features:
year_feature=['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']

for feature in year_feature:
    dataset[feature]=dataset['YrSold']-dataset[feature]

In [10]:
#To show all the rows of the dataset.
pd.pandas.set_option('display.max_columns',None)
dataset.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street    Alley LotShape  \
0   1          60       RL         65.0     8450   Pave  Missing      Reg   
1   2          20       RL         80.0     9600   Pave  Missing      Reg   
2   3          60       RL         68.0    11250   Pave  Missing      IR1   
3   4          70       RL         60.0     9550   Pave  Missing      IR1   
4   5          60       RL         84.0    14260   Pave  Missing      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1         Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3         Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4         Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     2Story            7            5          5   
1       Norm     1Fam     1Story            6            8         31   
2       Norm     1Fam     2Story            7            5          7   
3       Norm     1Fam     2Story            7            5         91   
4       Norm     1Fam     2Story            8            5          8   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0             5     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1            31     Gable  CompShg     MetalSd     MetalSd       None   
2             6     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3            36     Gable  CompShg     Wd Sdng     Wd Shng       None   
4             8     Gable  CompShg     VinylSd     VinylSd    BrkFace   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0       196.0        Gd        TA      PConc       Gd       TA           No   
1         0.0        TA        TA     CBlock       Gd       TA           Gd   
2       162.0        Gd        TA      PConc       Gd       TA           Mn   
3         0.0        TA        TA     BrkTil       TA       Gd           No   
4       350.0        Gd        TA      PConc       Gd       TA           Av   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          GLQ         706          Unf           0        150          856   
1          ALQ         978          Unf           0        284         1262   
2          GLQ         486          Unf           0        434          920   
3          ALQ         216          Unf           0        540          756   
4          GLQ         655          Unf           0        490         1145   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        Ex          Y      SBrkr       856       854             0   
1    GasA        Ex          Y      SBrkr      1262         0             0   
2    GasA        Ex          Y      SBrkr       920       866             0   
3    GasA        Gd          Y      SBrkr       961       756             0   
4    GasA        Ex          Y      SBrkr      1145      1053             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0       1710             1             0         2         1             3   
1       1262             0             1         2         0             3   
2       1786             1             0         2         1             3   
3       1717             1             0         1         0             3   
4       2198             1             0         2         1             4   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          Gd             8        Typ           0     Missing   
1             1          TA             6        Typ           1          TA   
2             1          Gd             6        Typ           1          TA   
3             1      

In [11]:
dataset[year_feature].head()

YearBuilt  YearRemodAdd  GarageYrBlt
0          5             5          5.0
1         31            31         31.0
2          7             6          7.0
3         91            36          8.0
4          8             8          8.0

In [12]:
# Handling the skewed values in numerical features
num_features=['LotFrontage', 'LotArea', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea',  'SalePrice']
for feature in num_features:
    if 0 in dataset[feature].unique():
        pass
    else:
        dataset[feature]=np.log(dataset[feature])

In [13]:
dataset[num_features].head()

LotFrontage   LotArea  TotalBsmtSF  1stFlrSF  GrLivArea  SalePrice
0     4.174387  9.041922          856  6.752270   7.444249  12.247694
1     4.382027  9.169518         1262  7.140453   7.140453  12.109011
2     4.219508  9.328123          920  6.824374   7.487734  12.317167
3     4.094345  9.164296          756  6.867974   7.448334  11.849398
4     4.430817  9.565214         1145  7.043160   7.695303  12.429216

### Handling rare categorical features:
We can remove the categories that are less than 1% of the total number of categories in categorical features.

In [14]:
cat_features=[features for features in dataset.columns if dataset[features].dtypes=='O']
cat_features

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [15]:
for feature in cat_features:
    temp=dataset.groupby(feature)['SalePrice'].count()/len(dataset)
    temp_df=temp[temp>0.01].index
    dataset[feature]=np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare_var')
   

In [16]:
dataset[cat_features]

MSZoning Street    Alley LotShape LandContour Utilities LotConfig  \
0          RL   Pave  Missing      Reg         Lvl    AllPub    Inside   
1          RL   Pave  Missing      Reg         Lvl    AllPub       FR2   
2          RL   Pave  Missing      IR1         Lvl    AllPub    Inside   
3          RL   Pave  Missing      IR1         Lvl    AllPub    Corner   
4          RL   Pave  Missing      IR1         Lvl    AllPub       FR2   
...       ...    ...      ...      ...         ...       ...       ...   
1455       RL   Pave  Missing      Reg         Lvl    AllPub    Inside   
1456       RL   Pave  Missing      Reg         Lvl    AllPub    Inside   
1457       RL   Pave  Missing      Reg         Lvl    AllPub    Inside   
1458       RL   Pave  Missing      Reg         Lvl    AllPub    Inside   
1459       RL   Pave  Missing      Reg         Lvl    AllPub    Inside   

     LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  \
0          Gtl      CollgCr       Norm       Norm     1Fam     2Story   
1          Gtl     Rare_var      Feedr       Norm     1Fam     1Story   
2          Gtl      CollgCr       Norm       Norm     1Fam     2Story   
3          Gtl      Crawfor       Norm       Norm     1Fam     2Story   
4          Gtl      NoRidge       Norm       Norm     1Fam     2Story   
...        ...          ...        ...        ...      ...        ...   
1455       Gtl      Gilbert       Norm       Norm     1Fam     2Story   
1456       Gtl       NWAmes       Norm       Norm     1Fam     1Story   
1457       Gtl      Crawfor       Norm       Norm     1Fam     2Story   
1458       Gtl        NAmes       Norm       Norm     1Fam     1Story   
1459       Gtl      Edwards       Norm       Norm     1Fam     1Story   

     RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType ExterQual  \
0        Gable  CompShg     VinylSd     VinylSd    BrkFace        Gd   
1        Gable  CompShg     MetalSd     MetalSd       None        TA   
2        Gable  CompShg     VinylSd     VinylSd    BrkFace        Gd   
3        Gable  CompShg     Wd Sdng     Wd Shng       None        TA   
4        Gable  CompShg     VinylSd     VinylSd    BrkFace        Gd   
...        ...      ...         ...         ...        ...       ...   
1455     Gable  CompShg     VinylSd     VinylSd       None        TA   
1456     Gable  CompShg     Plywood     Plywood      Stone        TA   
1457     Gable  CompShg     CemntBd     CmentBd       None        Ex   
1458       Hip  CompShg     MetalSd     MetalSd       None        TA   
1459     Gable  CompShg     HdBoard     HdBoard       None        Gd   

     ExterCond Foundation BsmtQual BsmtCond BsmtExposure BsmtFinType1  \
0           TA      PConc       Gd       TA           No          GLQ   
1           TA     CBlock       Gd       TA           Gd          ALQ   
2           TA      PConc       Gd       TA           Mn          GLQ   
3           TA     BrkTil       TA       Gd           No          ALQ   
4           TA      PConc       Gd       TA           Av          GLQ   
...        ...        ...      ...      ...          ...          ...   
1455        TA      PConc       Gd       TA           No          Unf   
1456        TA     CBlock       Gd       TA           No          ALQ   
1457        Gd   Rare_var       TA       Gd           No          GLQ   
1458        TA     CBlock       TA       TA           Mn          GLQ   
1459        TA     CBlock       TA       TA           No          BLQ   

     BsmtFinType2 Heating HeatingQC CentralAir Electrical KitchenQual  \
0             Unf    GasA        Ex          Y      SBrkr          Gd   
1             Unf    GasA        Ex          Y      SBrkr          TA   
2             Unf    GasA        Ex          Y      SBrkr          Gd   
3             Unf    GasA        Gd          Y      SBrkr          Gd   
4             Unf    GasA        Ex          Y      SBrkr          Gd   
...           ...     ...       ...        ...        ...  

In [19]:
for feature in cat_features:
    labels_ordered=dataset.groupby([feature])['SalePrice'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    dataset[feature]=dataset[feature].map(labels_ordered)

In [22]:
dataset[cat_features].head()

MSZoning  Street  Alley  LotShape  LandContour  Utilities  LotConfig  \
0         3       1      2         0            1          1          0   
1         3       1      2         0            1          1          2   
2         3       1      2         1            1          1          0   
3         3       1      2         1            1          1          1   
4         3       1      2         1            1          1          2   

   LandSlope  Neighborhood  Condition1  Condition2  BldgType  HouseStyle  \
0          0            14           2           1         3           5   
1          0            11           1           1         3           3   
2          0            14           2           1         3           5   
3          0            16           2           1         3           5   
4          0            22           2           1         3           5   

   RoofStyle  RoofMatl  Exterior1st  Exterior2nd  MasVnrType  ExterQual  \
0          0         0           10           10           2          2   
1          0         0            4            3           1          1   
2          0         0           10           10           2          2   
3          0         0            2            4           1          1   
4          0         0           10           10           2          2   

   ExterCond  Foundation  BsmtQual  BsmtCond  BsmtExposure  BsmtFinType1  \
0          3           4         3         3             1             6   
1          3           2         3         3             4             4   
2          3           4         3         3             2             6   
3          3           1         2         4             1             4   
4          3           4         3         3             3             6   

   BsmtFinType2  Heating  HeatingQC  CentralAir  Electrical  KitchenQual  \
0             5        2          4           1           3            2   
1             5        2          4           1           3            1   
2             5        2          4           1           3            2   
3             5        2          3           1           3            2   
4             5        2          4           1           3            2   

   Functional  FireplaceQu  GarageType  GarageFinish  GarageQual  GarageCond  \
0           4            1           4             2           2           3   
1           4            3           4             2           2           3   
2           4            3           4             2           2           3   
3           4            4           2             1           2           3   
4           4            3           4             2           2           3   

   PavedDrive  PoolQC  Fence  MiscFeature  SaleType  SaleCondition  
0           2       0      4            2         2              3  
1           2       0      4            2         2              3  
2           2       0      4            2         2              3  
3           2       0      4            2         2              0  
4           2       0      4            2         2              3

## Feature Scaling:

In [20]:
# Selecting the feature that should undergo feature scaling
features_scale=[features for features in dataset.columns if features not in ['Id','SalePrice']]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(dataset[features_scale])

MinMaxScaler()

In [23]:
scaler.transform(dataset[features_scale])

array([[0.23529412, 0.75      , 0.41820812, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.49506375, ..., 0.        , 0.        ,
        0.        ],
       [0.23529412, 0.75      , 0.434909  , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.29411765, 0.75      , 0.42385922, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.434909  , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.47117546, ..., 0.        , 0.        ,
        0.        ]])

In [26]:
# Transforming the dataset and adding Id and SalePrice features.
data=pd.concat([dataset[['Id','SalePrice']].reset_index(drop=True),
                pd.DataFrame(scaler.transform(dataset[features_scale]),columns=features_scale)],axis=1)

In [27]:
data.head()

Id  SalePrice  MSSubClass  MSZoning  LotFrontage   LotArea  Street  Alley  \
0   1  12.247694    0.235294      0.75     0.418208  0.366344     1.0    1.0   
1   2  12.109011    0.000000      0.75     0.495064  0.391317     1.0    1.0   
2   3  12.317167    0.235294      0.75     0.434909  0.422359     1.0    1.0   
3   4  11.849398    0.294118      0.75     0.388581  0.390295     1.0    1.0   
4   5  12.429216    0.235294      0.75     0.513123  0.468761     1.0    1.0   

   LotShape  LandContour  Utilities  LotConfig  LandSlope  Neighborhood  \
0  0.000000     0.333333        1.0       0.00        0.0      0.636364   
1  0.000000     0.333333        1.0       0.50        0.0      0.500000   
2  0.333333     0.333333        1.0       0.00        0.0      0.636364   
3  0.333333     0.333333        1.0       0.25        0.0      0.727273   
4  0.333333     0.333333        1.0       0.50        0.0      1.000000   

   Condition1  Condition2  BldgType  HouseStyle  OverallQual  OverallCond  \
0         0.4         1.0      0.75         1.0     0.666667        0.500   
1         0.2         1.0      0.75         0.6     0.555556        0.875   
2         0.4         1.0      0.75         1.0     0.666667        0.500   
3         0.4         1.0      0.75         1.0     0.666667        0.500   
4         0.4         1.0      0.75         1.0     0.777778        0.500   

   YearBuilt  YearRemodAdd  RoofStyle  RoofMatl  Exterior1st  Exterior2nd  \
0   0.036765      0.098361        0.0       0.0          1.0          1.0   
1   0.227941      0.524590        0.0       0.0          0.4          0.3   
2   0.051471      0.114754        0.0       0.0          1.0          1.0   
3   0.669118      0.606557        0.0       0.0          0.2          0.4   
4   0.058824      0.147541        0.0       0.0          1.0          1.0   

   MasVnrType  MasVnrArea  ExterQual  ExterCond  Foundation  BsmtQual  \
0        0.50     0.12250   0.666667        1.0        1.00      0.75   
1        0.25     0.00000   0.333333        1.0        0.50      0.75   
2        0.50     0.10125   0.666667        1.0        1.00      0.75   
3        0.25     0.00000   0.333333        1.0        0.25      0.50   
4        0.50     0.21875   0.666667        1.0        1.00      0.75   

   BsmtCond  BsmtExposure  BsmtFinType1  BsmtFinSF1  BsmtFinType2  BsmtFinSF2  \
0      0.75          0.25      1.000000    0.125089      0.833333         0.0   
1      0.75          1.00      0.666667    0.173281      0.833333         0.0   
2      0.75          0.50      1.000000    0.086109      0.833333         0.0   
3      1.00          0.25      0.666667    0.038271      0.833333         0.0   
4      0.75          0.75      1.000000    0.116052      0.833333         0.0   

   BsmtUnfSF  TotalBsmtSF  Heating  HeatingQC  CentralAir  Electrical  \
0   0.064212     0.140098      1.0       1.00         1.0         1.0   
1   0.121575     0.206547      1.0       1.00         1.0         1.0   
2   0.185788     0.150573      1.0       1.00         1.0         1.0   
3   0.231164     0.123732      1.0       0.75         1.0         1.0   
4   0.209760     0.187398      1.0       1.00         1.0         1.0   

   1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  \
0  0.356155  0.413559           0.0   0.577712      0.333333           0.0   
1  0.503056  0.000000           0.0   0.470245      0.000000           0.5   
2  0.383441  0.419370           0.0   0.593095      0.333333           0.0   
3  0.399941  0.366102           0.0   0.579157      0.333333           0.0   
4  0.466237  0.509927           0.0   0.666523      0.333333           0.0   

   FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  KitchenQual  TotRmsAbvGrd  \
0  0.666667       0.5         0.375      0.333333     0.666667      0.500000   
1  0.666667       0.0         0.375      0.333333     0.333333      0.333333   
2  0.666667       0.5         0.375      0.333333     0.666667      0.3

In [28]:
# Converting the dataset into a new csv file:
data.to_csv('x_train.csv')